In [ ]:
!pip install huggingface_hub
from huggingface_hub import login
login()

In [ ]:
import pandas as pd

df = pd.read_csv("hf://datasets/manueltonneau/arabic-hate-speech-superset/ar_hf_112024.csv")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df['dataset'].unique()

array(['L-HSAB', 'jhsc', 'T-HSAB', 'MLMA', 'saudi', 'aracovid',
       'alsafari', 'brothers', 'Let-Mi', 'OSACT'], dtype=object)

In [ ]:
Saudi_df=df[df['dataset'] == 'saudi']

In [ ]:
Saudi_df["label"] = Saudi_df["labels"].map({0.0: "not", 1.0: "offensive"})

<ipython-input-6-3960a665f4ed>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Saudi_df["label"] = Saudi_df["labels"].map({0.0: "not", 1.0: "offensive"})


In [ ]:
Saudi_df['label'].value_counts()

,count
label,
not,2328
offensive,850


In [ ]:
import re

def clean_text(text):
    text = re.sub(r'@[\w_]+', '', text)        # Remove @mentions
    text = re.sub(r'#\S+', '', text)           # Remove hashtags
    text = re.sub(r'\n', ' ', text)            # Replace line breaks with space
    text = re.sub(r'[^\w\s\u0600-\u06FF]', '', text)  # Remove punctuation (keep Arabic)
    text = re.sub(r'\s+', ' ', text).strip()   # Normalize spaces
    return text

Saudi_df["tweet"] = Saudi_df["text"].astype(str).apply(clean_text)


<ipython-input-7-bdcaca6d90e8>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Saudi_df["tweet"] = Saudi_df["text"].astype(str).apply(clean_text)


In [ ]:
Saudi_df.to_csv("Saudi.csv")

In [ ]:

# Filter by class to rebalance the datastet
offensive = df[df["label"] == "offensive"].sample(n=800, random_state=42)
not_offensive = df[df["label"] == "not"].sample(n=800, random_state=42)


balanced_df = pd.concat([offensive, not_offensive]).sample(frac=1, random_state=42).reset_index(drop=True)


balanced_df.to_csv("Saudi_balanced.csv", index=False)
